# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# Importing Libraries
import os
from dotenv import load_dotenv
import json
import sys
import io
import traceback
from openai import OpenAI

import tempfile
import base64

import gradio as gr

#Load the API key from the environment variable
load_dotenv(override=True)

# Check if we have a key
if os.environ.get("OPENAI_API_KEY"):
    print("API key found!")
else:
    print("No API key found! Please set OPENAI_API_KEY")
    
client = OpenAI()

In [ ]:
# Set up the tool- Execute Python code to compute results
def execute_python(code: str) -> str:
    """
    Safely execute Python code and return the output.
    This is our TOOL that the AI can call!
    """
    # Capture stdout so we can return it
    old_stdout = sys.stdout
    sys.stdout = buffer = io.StringIO()
    
    try:
        # Execute the code
        exec(code, {"__builtins__": __builtins__})
        output = buffer.getvalue()
        if not output:
            output = "Code ran successfully with no output."
        return output
    except Exception as e:
        return f"Error: {traceback.format_exc()}"
    finally:
        sys.stdout = old_stdout

# Define the tool schema
tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_python",
            "description": "Execute Python code and return the output. Use this to demonstrate code examples, run calculations, or test solutions to technical questions.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The Python code to execute"
                    }
                },
                "required": ["code"]
            }
        }
    }
]

# Quick test of the tool
test_result = execute_python("print('Hello from the tool!')\nprint(2 + 2)")
print("Tool test:", test_result)
print("Tool is working!")

In [ ]:

# Setting the context for the LLM
SYSTEM_PROMPT = """
You are a highly experienced software engineer. You specialize in Python, JavaScript, algorithms, system design, 
and general programming concepts.

Your personality:
- You explain things clearly, like a patient teacher
- You use analogies and examples to make complex topics accessible  
- You're enthusiastic about technology but not condescending
- When relevant, you DEMONSTRATE concepts by running actual code using your execute_python tool
- You always mention if there are gotchas, edge cases, or common mistakes to avoid

When someone asks a technical question:
1. Give a clear explanation first
2. If it involves code or calculations, USE the execute_python tool to show a working example
3. Explain what the code does and why
4. Mention any important caveats

"""

# Chat function with streaming and tools
def chat_with_streaming(message: str, history: list, model: str):
    """
    Send a message and get a streaming response.
    Handles tool calls too!
    
    history format: list of [user_message, assistant_message] pairs (Gradio format)
    """
    # Convert Gradio history format to OpenAI messages format
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    
    for human_msg, ai_msg in history:
        messages.append({"role": "user", "content": human_msg})
        if ai_msg:  # ai_msg might be None if still streaming
            messages.append({"role": "assistant", "content": ai_msg})
    
    # Add the new user message
    messages.append({"role": "user", "content": message})
    
    response_text = ""
    
    # First, try with streaming. When tools might be needed, 
    # we do a non-streaming first pass to check for tool calls.
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto"  # Let the AI decide when to use tools
    )
    
    # Check if the LLM infers tool usage
    if response.choices[0].finish_reason == "tool_calls":
        # 
        tool_call = response.choices[0].message.tool_calls[0]
        tool_args = json.loads(tool_call.function.arguments)
        
        # Show user we're executing code
        yield f" *Running code...*\n```python\n{tool_args['code']}\n```\n\n"
        
        # Run the code
        tool_result = execute_python(tool_args["code"])
        
        # Add tool results to messages and get final response
        messages.append(response.choices[0].message)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": tool_result
        })
        
        # Get the final streaming response with the tool results incorporated
        prefix = f" *Running code...*\n```python\n{tool_args['code']}\n```\n\n**Output:**\n```\n{tool_result}\n```\n\n"
        yield prefix
        
        # Stream the final explanation
        stream = client.chat.completions.create(
            model=model,
            messages=messages,
            stream=True
        )
        
        full_response = prefix
        for chunk in stream:
            if chunk.choices[0].delta.content:
                full_response += chunk.choices[0].delta.content
                yield full_response
    
    else:
        # No tool needed. Streaming the regular response
        stream = client.chat.completions.create(
            model=model,
            messages=messages,
            stream=True
        )
        
        for chunk in stream:
            if chunk.choices[0].delta.content:
                response_text += chunk.choices[0].delta.content
                yield response_text

print("Chat function working!")

In [ ]:
# Audio functionality
def transcribe_audio(audio_file_path: str) -> str:
    """
    Convert speech to text using Whisper.
    audio_file_path: path to the audio file from Gradio
    """
    if audio_file_path is None:
        return ""
    
    try:
        with open(audio_file_path, "rb") as audio_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file
            )
        return transcript.text
    except Exception as e:
        print(f"Transcription error: {e}")
        return f"[Audio transcription failed: {str(e)}]"

def text_to_speech(text: str) -> str | None:
    """
    Convert text to speech using OpenAI TTS.
    Returns path to audio file, or None if it fails.
    """
    # Cleaning up the text - removing markdown formatting for audio
    clean_text = text
    for char in ["**", "*", "```python", "```", "#", "`"]:
        clean_text = clean_text.replace(char, "")
    
    # Limit length for TTS
    if len(clean_text) > 1000:
        clean_text = clean_text[:1000] + "... [response truncated for audio]"
    
    try:
        with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as tmp_file:
            tmp_path = tmp_file.name
        with client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice="alloy",  # Options: alloy, echo, fable, onyx, nova, shimmer
            input=clean_text
        ) as response:
            response.stream_to_file(tmp_path)
        return tmp_path
    except Exception as e:
        print(f"TTS error: {e}")
        return None

print("Audio functions working!")

In [ ]:

# Gradio UI
# Handler functions that connect UI to logic

def handle_text_message(message: str, history: list, model: str):
    """
    Handle a text message - streams response back to chat.
    """
    if not message.strip():
        yield history, None
        return
    
    # Add user message to history
    history = history + [[message, None]]
    
    # Stream the response
    for partial_response in chat_with_streaming(message, history[:-1], model):
        history[-1][1] = partial_response
        yield history, None  # No audio yet during streaming
    
    # Generate audio for the final response
    final_response = history[-1][1]
    if final_response:
        audio_path = text_to_speech(final_response)
        yield history, audio_path


def handle_audio_message(audio_path: str, history: list, model: str):
    """
    Handle an audio message - transcribe it, then chat.
    """
    if audio_path is None:
        yield history, None
        return
    
    # Transcribe the audio
    transcribed_text = transcribe_audio(audio_path)
    
    if not transcribed_text:
        yield history, None
        return
    
    # Handling the audio message with text
    yield from handle_text_message(f" {transcribed_text}", history, model)


def clear_chat():
    """Reset everything"""
    return [], None, None


# Building the UI
with gr.Blocks(
    title="Technical Q&A Assistant",
    theme=gr.themes.Soft(),
    css="""
        .gradio-container { max-width: 900px; margin: auto; }
        .title-text { text-align: center; }
    """
) as demo:
    
    # Header
    gr.Markdown("""
    # Technical Q&A Assistant
    ### Your expert coding companion - ask me anything about programming!
    
    *I can explain concepts, write and run code examples, and even respond with audio!*
    """)
    
    with gr.Row():
        # Model selector
        model_dropdown = gr.Dropdown(
            choices=[
                "gpt-4o-mini",  # Faster and cheaper - good for most questions
                "gpt-4o",       # More powerful - for complex stuff
            ],
            value="gpt-4o-mini",
            label="Model",
            info="gpt-4o-mini is faster & cheaper; gpt-4o is smarter",
            scale=2
        )
    
    # Main chat area
    chatbot = gr.Chatbot(
        label="Chat",
        height=350,
        render_markdown=True,
        bubble_full_width=False,
        show_copy_button=True,
        avatar_images=(None, " ")  # User gets default, AI gets robot emoji
    )
    
    # Audio output (plays the AI's response)
    audio_output = gr.Audio(
        label="AI Voice Response",
        autoplay=True,
        visible=True
    )
    
    # Text input row
    with gr.Row():
        text_input = gr.Textbox(
            placeholder="Ask a technical question... e.g. 'How does recursion work?' or 'Show me how to sort a list in Python'",
            label="Your Question",
            lines=2,
            scale=5
        )
        send_btn = gr.Button("Send ", variant="primary", scale=1)
    
    # Audio input row
    with gr.Row():
        audio_input = gr.Audio(
            sources=["microphone"],
            type="filepath",
            label="🎤 Or speak your question!",
            scale=4
        )
        audio_send_btn = gr.Button("Send Audio 🎤", variant="secondary", scale=1)
    
    # Clear button
    clear_btn = gr.Button("Clear Chat", variant="stop")
    
    # Example questions to get people started
    gr.Examples(
        examples=[
            "What is a Python decorator and how do I use one?",
            "What is the significance of Transformers in AI",
            "What are Vector Embeddings?",
            "What's the difference between async and sync code?",
            "Write and run a fibonacci sequence in Python",
            "What is backpropagation in neural networks?",
        ],
        inputs=text_input,
        label="Example Questions (click to try!)"
    )
    
    # Wire up the events
    
    # Text message: submit on Enter or button click
    text_input.submit(
        fn=handle_text_message,
        inputs=[text_input, chatbot, model_dropdown],
        outputs=[chatbot, audio_output]
    ).then(lambda: "", outputs=text_input)  # Clear input after sending
    
    send_btn.click(
        fn=handle_text_message,
        inputs=[text_input, chatbot, model_dropdown],
        outputs=[chatbot, audio_output]
    ).then(lambda: "", outputs=text_input)
    
    # Audio message
    audio_send_btn.click(
        fn=handle_audio_message,
        inputs=[audio_input, chatbot, model_dropdown],
        outputs=[chatbot, audio_output]
    )
    
    # Clear chat
    clear_btn.click(
        fn=clear_chat,
        outputs=[chatbot, audio_output, audio_input]
    )

print("Gradio app working!")

In [ ]:
# Run the app!
demo.launch(
    debug=False,       # Set to True to see error messages in notebook
    show_error=True,   # Show errors in the UI
    inbrowser=True
)